In [128]:
import nistrng as nst
from nistrng.sp800_22r1a import *
import random
import numpy as np
import pandas as pd

In [95]:
#given a list of strings that represent random hex values, return an array of array's of the bits for each hex value
def true_rand_bin(hex):
    bin = []
    input = []
    for h in hex:
        b = "{0:08b}".format(int(h, 16))
        bin.append(b)
        temp = []
        for i in b:
            temp.append(int(i))
        input.append(np.array(temp))
    return input

In [96]:
hex_true_rand = ["e3bf00dc0aad2d180da829e5c620c6046fd726e36a1b2336e1067598ba307549", "f82897fe6abf801a7354271dc31e0b88e4f5da26bd4d64ac5076de173946a95c", "d3587c8bebd2b4e89626b4eca0d4b044256e4691610a6eb93aec7115249c58b5"]
true_rand_in = true_rand_bin(hex_true_rand)

In [125]:
def run_tests(inputs):
    scores = []
    for num in inputs:
        eligible_battery: dict = nst.check_eligibility_all_battery(num, SP800_22R1A_BATTERY)
        results = nst.run_all_battery(num, eligible_battery, False)
        for result, elasped_time in results:
            if result.passed:
                new_row = (result.name, "Passed", result.score, elasped_time)
                scores.append(new_row)
            else: 
                new_row = (result.name, "Passed", result.score, elasped_time)
                scores.append(new_row)
    return scores

In [127]:
results = run_tests(true_rand_in)

[('Monobit', 'Passed', 0.21129954733371056, 0), ('Frequency Within Block', 'Passed', 0.8155562560569334, 0), ('Runs', 'Passed', 0.7802088194570171, 0), ('Longest Run Ones In A Block', 'Passed', 0.008903527038229532, 0), ('Discrete Fourier Transform', 'Passed', 0.8185458083820408, 1), ('Non Overlapping Template Matching', 'Passed', 0.8948354728334884, 1), ('Serial', 'Passed', 0.5172141269096935, 12), ('Approximate Entropy', 'Passed', 0.12735780852635734, 5), ('Cumulative Sums', 'Passed', 0.2693873127926786, 0), ('Random Excursion', 'Passed', 0.19595759208921693, 3), ('Random Excursion Variant', 'Passed', 0.25668104117981316, 0), ('Monobit', 'Passed', 0.8025873486341526, 0), ('Frequency Within Block', 'Passed', 0.8441185338473761, 0), ('Runs', 'Passed', 0.7995200838206925, 0), ('Longest Run Ones In A Block', 'Passed', 0.4056298971046328, 0), ('Discrete Fourier Transform', 'Passed', 0.8185458083820408, 1), ('Non Overlapping Template Matching', 'Passed', 0.857123460498547, 0), ('Serial', '